This notebook contains the different machine learning models on which the training data of the <i>food-101</i> dataset will be trained on. Algorithms with different complexities will be used here. Its always a good practice to start with simplest model and later trying complex ones. But before we get our hands dirty with modeling, one more step lies between EDA and modeling which is Feature Engineering. In a usual scenario, feature engineering should get its separate notebook but because the dataset is already clean, images are already arranged in proper folders, all food items have 1000 images(except for one data object as seen in EDA), and data is well split into training and test set so, there is not much to do in feature engineering. Also, if we have to make some changes in the dataset it might be based on the model we choose. 

Starting the first model.
# Support Vector Machine 

Support vector machine is discriminative classifier formally defined by a separating hyperplane.

<i>SVM</i> is one of the simples models that we can you for classification. Images of different size could impact the learning of <i>SVM</i>. However, this is just an assumption. To see if this assumption holds we can train SVM on 2 datasets and evaluate the performance. To achieve this let's create a copy dataset where all the images are stored as square and of size <b>300x300</b>. 

### Feature engineering for SVM

Converting the rectangular images to square can be achieved through two ways. Either by shrinking the dimensions or cutting them out. Resizing the dimension will keep all the information but will move the image away from real world example. For example, let's see how the smallest image in the dataset will look like if we resize it to be square. 

<b>Original Image</b>

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline

image = Image.open('../../data/raw/food-101/images/macarons/3247436.jpg')
plt.imshow(image)
plt.show()

<b>Resized Image</b>

In [ ]:
import numpy as np

# Taking square root of the length * breath
sqrWidth = np.ceil(np.sqrt(image.size[0] * image.size[1])).astype(int) 
im_resize = image.resize((sqrWidth, sqrWidth))
plt.imshow(im_resize)
plt.show()

This looks quite bad but still holds the information about the food. Let's see what happens when we cut the extra dimensions to make the image square.

In [ ]:
# Create a new square white image with dimenion equal to smaller side of original image
# then paste the original image over the white image
def make_square(image, max_size=150, fill_color=(0, 0, 0, 0)):
    x, y = image.size
    size = min(max_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(image, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

new_image = make_square(image)
plt.imshow(new_image)
plt.show()

This looks more like a real image, infact this image removes noise from the original image. However, we lose information while cropping the image. 

For the later method of cropping an image we can do a little variation and create a new kind of square image. Instead of using the smaller side of the image, we can use the longer one and fill the extra space with black or white color to generate a square image.

In [ ]:
# Create a new square white image with dimenion equal to smaller side of original image
# then paste the original image over the white image
def make_big_square(image, min_size=256, fill_color=(0, 0, 0)):
    x, y = image.size
    size = max(min_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(image, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

new_image = make_square(image)
plt.imshow(new_image)
plt.show()

This keeps all the information and convert the image to square but adds lot more information. We don't know whether this helps with learning or not. We can create new dataset of images in this format as well.

Things to remember:
* check the number of images with ratio more than 2:1
* images where we will lose the information if we crop to make it rectangular
* using an algorithm to find the important part of food image which can be used to check the performance as well
* after making it square, check performance with or without making all images of same size
* use 5 different classifier atleast to know the performance